# Exploratory Data Analysis: Correlations

The purpose of this notebook will be to walkthrough various correlation analyses with a focus on lagging price across permits to see any relationship between permits and pricing. Our expectation is that if permits increase supply, we will see a negative relationship between permits and price. However, we might also encounter a situation where there is no discernable relationship or that the relationship is the opposite of our expectation due to demand driven exogenous features that need to be controlled for before we can truely understand the relationship between permits and prices.

In [75]:
#Import necessary packages and connect to google drive
import numpy as np
import pandas as pd
import altair as alt

In [76]:
#Google drive pathway needs to be edited based on user's specific pathway to shared repository
permits = pd.read_parquet('data/permits_final.parquet.gzip')
price = pd.read_parquet('data/price_final.parquet.gzip')

permits.head()

measure                              county_name  region_code  division_code  \
DATE       FIPS                                                                
2000-01-01 01001  Autauga County                            3              6   
           01081  Lee County                                3              6   
           01113  Russell County                            3              6   
           01125  Tuscaloosa County                         3              6   
           02013  Aleutians East Borough                    4              9   

measure           bldgs  units    value  
DATE       FIPS                          
2000-01-01 01001     13     13   690525  
           01081     32     63  5558536  
           01113      3      4   343000  
           01125     56     60  5353849  
           02013      0      0        0

In [77]:
#Joining permits and price based on index
permits_price = permits.join(price, how='inner')
permits_price.head()

county_name  region_code  division_code  \
DATE       FIPS                                                                
2012-01-01 01001  Autauga County                            3              6   
           01007  Bibb County                               3              6   
           01009  Blount County                             3              6   
           01021  Chilton County                            3              6   
           01037  Coosa County                              3              6   

                  bldgs  units    value  INVENTORY  HOMES_SOLD  SALE_PRICE  \
DATE       FIPS                                                              
2012-01-01 01001      9      9  2107491      286.0          20      133250   
           01007      1      1   163303       24.0           0      186650   
           01009      1      1   350000      131.0          19       68876   
           01021      2      2   254433      162.0           9      128900   
           01037      0      0        0       11.0           1      115000   

                  SALE_PRICE_ADJ  LIST_PRICE  LIST_PRICE_ADJ  
DATE       FIPS                                               
2012-01-01 01001          133250    157450.0        157450.0  
           01007          186650     99900.0         99900.0  
           01009           68876    132000.0        132000.0  
           01021          128900    111200.0        111200.0  
           01037          115000    142900.0        142900.0

In [78]:
permits_price.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 204752 entries, (Timestamp('2012-01-01 00:00:00'), '01001') to (Timestamp('2025-03-01 00:00:00'), '56045')
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   county_name     204752 non-null  object 
 1   region_code     204752 non-null  int64  
 2   division_code   204752 non-null  int64  
 3   bldgs           204752 non-null  int64  
 4   units           204752 non-null  int64  
 5   value           204752 non-null  int64  
 6   INVENTORY       201032 non-null  float64
 7   HOMES_SOLD      204752 non-null  int64  
 8   SALE_PRICE      204752 non-null  int64  
 9   SALE_PRICE_ADJ  204752 non-null  int64  
 10  LIST_PRICE      203876 non-null  float64
 11  LIST_PRICE_ADJ  203876 non-null  float64
dtypes: float64(3), int64(8), object(1)
memory usage: 19.6+ MB


### Lagged Correlations

To start, we will look at the correlation of the normalized results of units and sale price. Since we are using data from counties with vastly different scales in terms of units and value, we want to make sure everything is evaluated on an equal basis.

Our first analysis looks at negative correlations. We want to see the frequencies of the most common lags for negative correlations. Then, we will look at the overall correlation by time period.

In [79]:
data = permits_price.copy().reset_index()

#Since we are going to compare across counties with differences (urban, rural, etc.), we will normalize our variables of interest
data['units_norm'] = data.groupby('county_name')['units'].transform(lambda x: (x - x.mean()) / x.std())
data['SALE_PRICE_NORM'] = data.groupby('county_name')['SALE_PRICE_ADJ'].transform(lambda x: (x - x.mean()) / x.std())

#Need to agg dataset for total cross corr later
monthly_data = data.groupby('DATE').agg({'units_norm': 'mean', 'SALE_PRICE_NORM': 'mean'})


data.dropna(subset=['units_norm', 'SALE_PRICE_NORM'], inplace=True)

monthly_data.head()

,units_norm,SALE_PRICE_NORM
DATE,,
2012-01-01,-0.622792,-1.083192
2012-02-01,-0.516945,-1.054281
2012-03-01,-0.352516,-0.938897
2012-04-01,-0.302479,-0.808841
2012-05-01,-0.187768,-0.761798


In [80]:
#Now that we have our norm columns, we can apply the lagged correlation by county
#https://medium.com/pythoneers/cross-correlation-and-coherence-in-time-series-analysis-how-to-uncover-relationships-between-c83a08990b2d

#This function takes in the level we want to group and the variables we want to correlate.
#It's important to note we are only looking at impact on future periods in this lagged analysis.

def get_corr_values(group_level, var1, var2, period_lag=24):

  best_corr = None
  best_lag = None

  for lag in range(-period_lag, 0):
      x = group_level[var1]
      y = group_level[var2].shift(lag)
      valid = pd.concat([x, y], axis=1).dropna() #Corr has warnings if not enough records

      #Only apply corr when we have records
      if len(valid) > 1:
          corr = valid[var1].corr(valid[var2])
          if pd.notnull(corr) and (best_corr is None or corr < best_corr):
              best_corr = corr
              best_lag = int(lag)

  return pd.Series(
      {
          'best_corr': best_corr,
          'best_lag': best_lag
      }
  )



corr_summary = data.groupby('county_name').apply(get_corr_values,
                                                 var1='units_norm',
                                                 var2='SALE_PRICE_NORM')

corr_summary.sort_values('best_corr', ascending=True)

,best_corr,best_lag
county_name,,
Tolland County,-1.000000,-17.0
La Porte County,-1.000000,-9.0
Upson County,-1.000000,-24.0
St. Mary's County,-1.000000,-22.0
Wallowa County,-1.000000,-15.0
...,...,...
Canyon County,0.711469,-24.0
St. Bernard Parish,0.808273,-2.0
Johnston County,0.816517,-17.0


Before we start, we will do some minor data cleaning to prep the results for our visuals.

In [81]:
#Flipping axis so time lag period can be read left to right
corr_summary['best_lag'] = corr_summary['best_lag'] * -1

#Filter out NaN values
corr_summary = corr_summary[corr_summary['best_corr'].notna()]

To begin, we will look at a couple of options to display lag density or count for all counties. This will display the lag with the highest correlation found in our function applied at a county group level.

In [82]:
#Using alt chart to display lines instead of density to improve the data to ink ratio and not hide the large values
alt.Chart(corr_summary).mark_bar(size=12, color='#1f77b4').encode(
    alt.X("best_lag:Q", bin=alt.Bin(step=1), axis=alt.Axis(grid=False, format='.0f'), title='Months From Observation'),
    alt.Y('count()', axis=alt.Axis(grid=False), title='Count')
).properties(
    width=800,
    height=300,
    title='Count of Best Lags'
)


alt.Chart(...)

In [83]:
corr_summary_36 = data.groupby('county_name').apply(get_corr_values,
                                                 var1='units_norm',
                                                 var2='SALE_PRICE_NORM',
                                                 period_lag=36)

#Flipping axis so time lag period can be read left to right
corr_summary_36['best_lag'] = corr_summary_36['best_lag'] * -1

#Filter out NaN values
corr_summary_36 = corr_summary_36[corr_summary_36['best_corr'].notna()]



In [84]:
#Using alt chart to display lines instead of density to improve the data to ink ratio and not hide the large values
alt.Chart(corr_summary_36).mark_bar(size=12, color='#1f77b4').encode(
    alt.X("best_lag:Q", bin=alt.Bin(step=1), axis=alt.Axis(grid=False, format='.0f'), title='Months From Observation'),
    alt.Y('count()', axis=alt.Axis(grid=False), title='Count')
).properties(
    width=800,
    height=300,
    title='Count of Best Lags'
)

alt.Chart(...)

Using corr_summary or corr_summary_36, both result in greater impact in future. This pattern seems indicative of not actual improvement, but rather underlying changes in the real estate markets.

For example, since both have best lags the further out we look, we should be suspect that what we are finding is an actual relationship between permits and price. While it is unclear what might be causing such a skew in the strength of our lagged correlation, it emphasizes that we would need to better control for exogenous variables and cannot solely rely on a correlation to give us any reasonable support behind our initial expectations.


### Total correlation by time period

Next we will analyze the overall correlation lag strength by plotting the strongest lag's correlation result over time. To do this, we simply input slight modification to our get_corr_values function to group at the DATE level. We do this by not grouping at all and just running the corr on the aggregated monthly data defined above

In [85]:
#Revised function for total corr without group level
def get_lagged_correlations(df, var1, var2, period_lag=24):
    results = {}
    for lag in range(-period_lag, 0):
        corr = df[var1].corr(df[var2].shift(lag))
        results[lag] = corr
    return pd.Series(results, name='correlation')


In [86]:
total_corr = get_lagged_correlations(monthly_data, var1="units_norm", var2="SALE_PRICE_NORM").reset_index().rename(columns={'index':'lag'})
total_corr['lag'] = total_corr['lag'] * -1
total_corr = total_corr.sort_values(by='lag', ascending=True)

total_corr.head()

,lag,correlation
23,1,0.514974
22,2,0.424434
21,3,0.374152
20,4,0.310799
19,5,0.253993


In [87]:
bars = alt.Chart(total_corr).mark_bar(size=5).encode(
    x=alt.X('lag:O',
            title='Months Lagged',
            axis=alt.Axis(labelAngle=0)),
    y=alt.Y('correlation:Q',
            title='Correlation'),
    color=alt.condition(
        'datum.correlation < 0',
        alt.value('#d95f02'),
        alt.value('#1f77b4')
    )
)

dots = alt.Chart(total_corr).mark_point(filled=True, size=100).encode(
    x=alt.X('lag:O',
            title='',
            axis=alt.Axis(labelAngle=0)),
    y=alt.Y('correlation:Q'),
    color=alt.condition(
        'datum.correlation < 0',
        alt.value('#d95f02'),
        alt.value('#1f77b4')
    )
)

chart = (bars + dots).properties(
    width=800,
    height=300,
    title='Dot Plot of Correlations Lagged Over 1-24 Months'
)

chart

alt.LayerChart(...)

Above we see that beyond the initial months, we do not see any consistently strong relationship between permits and price at the aggregate date level. Even the strongest lagged values result in roughly a 0.20 - 0.50 correlation for 6 months out to 1 month out, which is not indicative of a very strong relationship.

With the strongest relationships being almost exclusively positive (i.e., increase in units leading to increase in price), we see the expected result that many factors are driving up price that need to be controlled for via causal analysis before we can obtain an understanding of permits on price.

As seen above, we have nearly all positive values with two negative values. A likely explaination could be found in simply rising prices in high demand areas (positive corr) or lowering pricing in low demand areas (negative corr) rather capturing any true impact of permits & development on pricing.

This seemingly demand driven environment would indicate the opposite kind of relationship than what we initially expected. Instead of permits and development functioning as a mechanism to reduce inequality, it appears that it may be a response to increasing demand along with various other demand driven factors.